# Combine DESI Imaging ccds

The eboss ccd files did not have the same dtype, therefore we could not easily combine them. We have to enfore a dtype to all of them.

In [1]:
# import modules
import fitsio as ft
import numpy as np
from glob import glob

In [2]:
# read files 
ccdsn = glob('/Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-*')
print(ccdsn)  # ccdfiles names


prt_keep = ['camera', 'filter', 'fwhm', 'mjd_obs', 'exptime', 
            'ra', 'dec', 'ra0','ra1','ra2','ra3','dec0','dec1','dec2','dec3',
           'galdepth', 'ebv', 'airmass', 'ccdskycounts', 'pixscale_mean', 'ccdzpt']

# read one file to check the columns
d = ft.read(ccdsn[0], columns=prt_keep)
print(d.dtype)

['/Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-90prime-new.fits', '/Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-decam-dr8.fits', '/Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-mosaic-dr8.fits']
[('camera', 'S7'), ('filter', 'S1'), ('exptime', '>f4'), ('mjd_obs', '>f8'), ('airmass', '>f4'), ('fwhm', '>f4'), ('ra', '>f8'), ('dec', '>f8'), ('ccdzpt', '>f4'), ('ccdskycounts', '>f4'), ('ra0', '>f8'), ('dec0', '>f8'), ('ra1', '>f8'), ('dec1', '>f8'), ('ra2', '>f8'), ('dec2', '>f8'), ('ra3', '>f8'), ('dec3', '>f8'), ('pixscale_mean', '>f4'), ('ebv', '>f4'), ('galdepth', '>f4')]


In [3]:
#  attrs for the general quicksip
#  'crval1', 'crval2', 'crpix1', 'crpix2', 'cd1_1',
#  'cd1_2', 'cd2_1', 'cd2_2', 'width', 'height'
# dtype = np.dtype([('filter', 'S1'), ('exptime', '>f4'), ('mjd_obs', '>f8'), ('airmass', '>f4'),\
#                   ('fwhm', '>f4'), ('width', '>i2'), ('height', '>i2'), ('crpix1', '>f4'), ('crpix2', '>f4'),\
#                   ('crval1', '>f8'), ('crval2', '>f8'), ('cd1_1', '>f4'), ('cd1_2', '>f4'), ('cd2_1', '>f4'),\
#                   ('cd2_2', '>f4'), ('ra', '>f8'), ('dec', '>f8'), ('ccdzpt', '>f4'), ('ccdskycounts', '>f4'),
#                   ('pixscale_mean', '>f4'), ('ebv', '>f4'), ('galdepth', '>f4')])
#
# only read & combine the following columns
# this is what the pipeline need to make the MJD maps

prt_keep = ['camera', 'filter', 'fwhm', 'mjd_obs', 'exptime', 
            'ra', 'dec', 'ra0','ra1','ra2','ra3','dec0','dec1','dec2','dec3',
           'galdepth', 'ebv', 'airmass', 'ccdskycounts', 'pixscale_mean', 'ccdzpt']

# camera could be different for 90prime, decam, mosaic -- we pick S7
dtype = np.dtype([('camera', 'S7'),('filter', 'S1'), ('exptime', '>f4'), ('mjd_obs', '>f8'), 
                  ('airmass', '>f4'), ('fwhm', '>f4'), ('ra', '>f8'), ('dec', '>f8'), ('ccdzpt', '>f4'),
                  ('ccdskycounts', '>f4'), ('ra0', '>f8'), ('dec0', '>f8'), ('ra1', '>f8'),
                  ('dec1', '>f8'), ('ra2', '>f8'), ('dec2', '>f8'), ('ra3', '>f8'), ('dec3', '>f8'),
                  ('pixscale_mean', '>f4'), ('ebv', '>f4'), ('galdepth', '>f4')])


def fixdtype(data_in, indtype=dtype):
    m = data_in.size
    data_out = np.zeros(m, dtype=dtype)
    for name in dtype.names:
        data_out[name] = data_in[name].astype(dtype[name])
    return data_out

In [4]:
#
# read each ccd file > fix its dtype > move on to the next
ccds_data = []
for ccd_i in ccdsn:
    print('working on .... %s'%ccd_i)
    data_in = ft.FITS(ccd_i)[1].read(columns=prt_keep)
    #print(data_in.dtype)
    data_out = fixdtype(data_in)
    print('number of ccds in this file         : %d'%data_in.size)
    print('number of different dtypes (before) : %d'%len(np.setdiff1d(dtype.descr, data_in.dtype.descr)), np.setdiff1d(dtype.descr, data_in.dtype.descr))
    print('number of different dtypes (after)  : %d'%len(np.setdiff1d(dtype.descr, data_out.dtype.descr)), np.setdiff1d(dtype.descr, data_out.dtype.descr))
    ccds_data.append(data_out)

working on .... /Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-90prime-new.fits
number of ccds in this file         : 145092
number of different dtypes (before) : 0 []
number of different dtypes (after)  : 0 []
working on .... /Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-decam-dr8.fits
number of ccds in this file         : 8675339
number of different dtypes (before) : 1 ['|S7']
number of different dtypes (after)  : 0 []
working on .... /Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-mosaic-dr8.fits
number of ccds in this file         : 241724
number of different dtypes (before) : 1 ['|S7']
number of different dtypes (after)  : 0 []


In [5]:
ccds_data_c = np.concatenate(ccds_data)
print('Total number of combined ccds : %d'%ccds_data_c.size)

Total number of combined ccds : 9062155


In [6]:
ft.write('/Volumes/TimeMachine/data/DR8/ccds/ccds-annotated-dr8_combined.fits',
        ccds_data_c, header=dict(NOTE='dr8 combined'), clobber=True)